In [21]:
import pandas as pd
buy_trades = pd.read_json('data/buy_trades_dec1.json', lines=True,)
sell_trades = pd.read_json('data/sell_trades_dec1.json', lines=True)

In [22]:
buy_trades.sort_values(by=["timestamp"], ascending=False, inplace=True)
buy_trades.drop_duplicates(subset=["advNo"], keep='first', inplace=True)
sell_trades.sort_values(by=["timestamp"], ascending=False, inplace=True)
sell_trades.drop_duplicates(subset=["advNo"], keep='first', inplace=True)

In [23]:
sell_trades = sell_trades[sell_trades["timestamp"] > pd.to_datetime('2022-11-15 11:59:59')]
buy_trades = buy_trades[buy_trades["timestamp"] > pd.to_datetime('2022-11-15 11:59:59')]

In [24]:
sell_trades["tradableVolumeInPrice"] = sell_trades["tradableQuantity"] * sell_trades["price"]
sell_trades["averageSingleTransAmount"] = sell_trades["maxSingleTransAmount"]-sell_trades["minSingleTransAmount"]
buy_trades["tradableVolumeInPrice"] = buy_trades["tradableQuantity"] * buy_trades["price"]
buy_trades["averageSingleTransAmount"] = buy_trades["maxSingleTransAmount"]-buy_trades["minSingleTransAmount"]


In [25]:
st = sell_trades[["asset", "price", "minSingleTransAmount", "maxSingleTransAmount",
 "tradableQuantity", "price", "tradableVolumeInPrice", "averageSingleTransAmount"]]
s = st.groupby(["asset"]).mean()
bt = buy_trades[["asset", "price", "minSingleTransAmount", "maxSingleTransAmount",
 "tradableQuantity", "price", "tradableVolumeInPrice", "averageSingleTransAmount"]]
b = bt.groupby(["asset"]).mean()

In [26]:
sc = sell_trades[["_id", "asset"]].groupby(["asset"]).count()
bc = buy_trades[["_id", "asset"]].groupby(["asset"]).count()

In [27]:
sc.rename({"_id": "sell_count"}, axis=1, inplace=True)
bc.rename({"_id": "buy_count"}, axis=1, inplace=True)

sn = s.rename({"price": "sell_price",
 "minSingleTransAmount": "sell_minSingleTransAmount", 
 "maxSingleTransAmount": "sell_maxSingleTransAmount",
 "tradableQuantity": "sell_tradableQuantity",
 "price": "sell_price",
 "tradableVolumeInPrice": "sell_tradableVolumeInPrice", 
 "averageSingleTransAmount": "sell_averageSingleTransAmount"}, axis=1, inplace=False)
bn = b.rename({"price": "buy_price",
 "minSingleTransAmount": "buy_minSingleTransAmount", 
 "maxSingleTransAmount": "buy_maxSingleTransAmount",
 "tradableQuantity": "buy_tradableQuantity",
 "price": "buy_price",
 "tradableVolumeInPrice": "buy_tradableVolumeInPrice", 
 "averageSingleTransAmount": "buy_averageSingleTransAmount"}, axis=1, inplace=False)

In [28]:
fin = pd.merge(sc, bc, on=["asset"], how="outer")
fin2 = pd.merge(sn, bn, on=["asset"], how="outer")

fina = pd.merge(fin, fin2, on=["asset"], how="outer")


In [29]:
fina.to_csv("csv/averages_on_assets_post_nov_15.csv")
